In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import re
import os
import math
from functools import reduce
import copy
import pickle

import uploading_lib as ulib
import mmx_lib as mmx
from tqdm.notebook import tqdm

from pytrends.request import TrendReq

## Загрузка данных

Для начала нам нужно обновить данные. По-хорошему, данные подгружаются каждый день, соответственно, нам нужно добавить данные лишь за один день.

Однако, если скрипт долго не запускался, для погоды будут подгружены только последние пять дней (те, что можно выгрузить бесплатно), если вы считаете, что нужно заплатить денег и подгрузить все недостающее, нужно пойти в бибилиотеку uploading_lib.py и в функции weather_upload() закомментировать строчку timestamps_to_upload = timestamps_to_upload[:5]

In [2]:
youscan = pd.read_csv('DATA/youscan_Trigger-MMX.csv')
youscan = ulib.youscan_upload(youscan)
print('saving')
youscan.to_csv('DATA/youscan_Trigger-MMX.csv', index=False)

/home/analyst/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2021-03-17 2021-03-17
saving


In [3]:
gt = ulib.gt_add_recent()
gt.to_csv('DATA/google-trends_Trigger-MMX_recent.csv')

In [4]:
weather = pd.read_csv('DATA/regions_weather.csv', header=0, index_col=0)
weather = ulib.weather_upload(weather)
weather.to_csv('DATA/regions_weather.csv')

In [5]:
geo = pd.read_csv('DATA/geo-rus.csv')

Нас инетерсуют тольк записи с датами публикации, мы хотим знать, в каком регионе была сделала публикация, а так же мы не хотим упоминания о коронавирусе (это не значит, что мы полностью избавляемся от эффекта паники весной 2020 года, мы просто по возможности исключаем совсем нерелевантные посты)

In [6]:
youscan = youscan[youscan['fullText'].notna()&youscan['published'].notna()]
youscan = youscan.merge(geo[['Город', 'Регион']], left_on='city', right_on='Город', how='left')
youscan_no_covid = youscan[~youscan['fullText'].str.contains(r'корона|коронавирус|каронавирус|кароновирус|ковид|covid|corona', case=False,  regex=True)]
print(len(youscan), len(youscan_no_covid))

465149 446052


## Предобработка данных

In [7]:
#Мы хотим, чтобы все индексы были в формате даты, для упрощения объединения и построения графиков
weather.index = [datetime.strptime(date+'-0', "%Y-%W-%w") for date in weather.index]
gt.index = [datetime.strptime(date, "%Y-%m-%d") for date in gt.index]

In [8]:
media = youscan_no_covid[['published', 'Регион', 'id']]
media = media.groupby(['published', 'Регион'])['id'].count().unstack().fillna(0)

media.index = [datetime.strptime(date.split('T')[0], "%Y-%m-%d") for date in media.index]
#Уберем возникшие дупликаты дат
media = media.groupby(level=0).sum()

#Сгруппировать до недель?
media_week = media.copy()
media_week['week'] = media_week.index.map(lambda x: datetime.strftime(x, '%Y-%W'))
media_week = media_week.groupby('week').sum()
media_week.index = [datetime.strptime(date+'-0', "%Y-%W-%w") for date in media_week.index]
media_week = media_week.groupby(level=0).sum()   


## Метрика качества

In [9]:
#При построении моделей мы также сохранили наиболее коррелирующие погодные метрики
with open('metric_dict.pickle', 'rb') as handle:
    metric_dict = pickle.load(handle)
    
g_list = ['Камчатский край', 'Сахалинская область']

In [10]:
def maximum_search(df, alpha, weeks, period=1):
    """Позволяет создавать признаки для модели и целевую переменную"""
    #сравним текущее значение со средними
    nearby_median_long = df.rolling(360, min_periods=30).median()
    nearby_median_short = df.rolling(60, min_periods=7).median()
    df_delta = ((df - nearby_median_long)/nearby_median_long>alpha)|((df - nearby_median_short)/nearby_median_short>alpha)

    #мы хотим отслеживать моменты, когда значения не слишком высоки, но тем не менее стабильно растут
    df_diff = df.diff()
    #df_growing = ((df_diff>0).rolling(7*period).sum()>=7*period)&((df - nearby_median_long)/nearby_median_long>0)
    df_growing = ((df_diff>0).rolling(7*(period+1)).sum()==7*(period+1))&((df - nearby_median_short)/nearby_median_short>0.5)
    df_delta = df_delta|df_growing

    #мы готовы допускать пропуски не более, чем в три дня (чем определено?)
    df_holes = mmx.find_range_of_holes(df_delta, (1,4))
    df_delta = df_delta|df_holes

    #мы не хотим пики длиной меньше недели
    df_short = mmx.find_range_of_holes(~df_delta, (1, (7*period)+1))
    df_delta = df_delta&(~df_short)

    #подсчет производных                
    df_diff = df_diff.rolling(window=7*period).mean()                
    df_diff = mmx.series_norm(df_diff)
    df_diff2 = df_diff.diff().rolling(window=7*period).mean()
    df_diff2 = mmx.series_norm(df_diff2)

    df_week = df.rolling(window=7*period).mean()/nearby_median_short

    #Почему так расставлена перменная weeks?
    #Потому что для двухнедельного прогноза мы хотим знать, 
    #что к концу второй недели начнется подъем активности, 
    #нас интересует именно конец этого диапазона

    df_target = (df_delta.shift(-7*period*weeks).rolling(window=7*period).sum()>3*period).astype(int)
    return (df_target, df_delta, df_diff, df_diff2, df_week)
    
def region_prediction(weeks=1, num=5):
    predictions = {}
    today = (datetime.now() + timedelta(hours=3)).date().strftime('%Y-%m-%d')
    
    for reg in tqdm(sorted(list(geo['Регион'].unique()))): 
#     for reg in tqdm(['Московская область', 'Пензенская область', 'Тверская область', 'Хабаровский край']):        
        #Мы будем пытаться строить предсказание только для тех регионов, 
        #для которых нам удалось натренировать модель
        models = os.listdir('models')
        models = [m for m in models if (m.startswith(reg+'_'))&(f'{weeks}week' in m)]
        if len(models)>0:
            predictions[reg] = np.nan
            #Сохраним себе названия файлов с моделями
            final_reg = [m for m in models if 'finalregr' in m][0]
            #Сортировка навзаний необходима, так как при обучении датасет итоговой регрессии
            #содержал столбцы с результатами ансамля в лексиграфическом порядке.
            estimators = sorted([m for m in models if ('finalregr' not in m)&('colnames' not in m)])
            colnames = [name for name in models if 'colnames' in name][0]
            #Флаг, котрый переключится на False, если у нас что-то поломалось в данных 
            #и датасет для региона не собрался
            flag=True
            #Флаг, который будет указывать на то, что основным источником данных НЕ является 
            #выгрузка соцмедиа данных из youscan
            m_flag=True
            if (reg in list(media.columns))&(reg not in g_list):
                m = media[reg]
                #Проверим средний объем данных (не учитывая этот год, так как он, во-первых, все поломал,
                #а, во-вторых, модели обучались на данных без учета 2020 года)
                this_year = datetime.fromisoformat('2020-01-01')
                m_check = m[m.index<this_year]
                m_check = m_check.sum()/len(m_check)
                #0.1 - экспериментально полученная величина, при которой еще можно построить предсказание
                if m_check>0.1:
                    #Проверка пройдень, модель строится на медийных данных, флаг переключился
                    m_flag = False
                    #Подгрузим остальные данные, если они есть
                    if reg in list(gt.columns):
                        g = gt[reg]
                    else:
                        g = pd.Series()

                    if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                        w = weather[weather['Регион']==reg][metric_dict[reg]]
                        w = w.groupby(level=0).mean()
                    else:
                        w = pd.Series()

                    #Мы не хотим учитывать слишком большие всплески.
                    std = m.std()
                    mean = m.mean()
                    m = m.apply(lambda x: min(x, mean+3*std))

                    #Несмотря на то, что мы берем разбивку по дням, лучше данные усреднить
                    m = m.rolling(window=14).mean()
                    #А также все отнормировать
                    m = mmx.series_norm(m)
                    w = mmx.series_norm(w)
                    g = mmx.series_norm(g)
                    #На всякий случай, при выгрузке иногда случаются дупликаты дат
                    g = g.groupby(level=0).mean()   

                    alpha = 0.3
                    #Для соцмедиа данных и для google trends посчитаем всякие полезные признаки
                    m_target, m_delta, m_diff, m_diff2, m_week = maximum_search(m, alpha, weeks, period=1)
                    g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=2)
                    #А если данных в регионе маловато, добавим к медийной целевой перменной gt
                    #if (m_check<1)&(reg in list(gt.columns)): 
                    if (m_check<1):                    
                        m_target = (m_target+g_target)>0
                        m = m+g
                        m = mmx.series_norm(m)
                    #Мы хотим собрать такой датасет только из того, что действительно есть                   
                    if (reg in list(weather['Регион']))&(reg in list(metric_dict.keys()))&(reg in list(gt.columns)):
                        x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                       w.shift(1), \
                                       g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                        feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                         'weather', \
                                         'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                    elif reg in list(gt.columns):
                        x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, \
                                       g_week, g_diff<0, g_diff>0, g_diff2, g_delta], axis=1)
                        feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                        'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2', 'gdelta']
                    elif (reg in list(weather['Регион']))&(reg in list(metric_dict.keys())):
                        x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta, w.shift(1)], axis=1)
                        feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta', \
                                         'weather']
                    else:
                        x = pd.concat([m_week, m_diff<0, m_diff>0, m_diff2, m_delta], axis=1)
                        feature_names = ['mweek', 'mdiff<0', 'mdiff>0', 'mdiff2', 'mdelta']

            #А вот что, если соцмедиа данных не хватает?
            if m_flag:
                #Пробуем строить на основе погоды и gt
                if ((reg in list(gt.columns))&((reg in list(weather['Регион']))&(reg in list(metric_dict.keys())))&m_flag)|(reg in g_list):
                    w = weather[weather['Регион']==reg][metric_dict[reg]]
                    w = w.groupby(level=0).mean()            
                    w = mmx.series_norm(w)
                    g = gt[reg]
                    g = mmx.series_norm(g)

                    alpha = 0.3
                    g_target, g_delta, g_diff, g_diff2, g_week = maximum_search(g, alpha, weeks, period=1)
                    x = pd.concat([w, w.rolling(4).mean(), w.diff(), g_delta, g_week, g_diff<0, g_diff>0, g_diff2], axis=1)
                    feature_names = ['weather', 'wmean', 'wdiff', 'gdelta', 'gweek', 'gdiff<0', 'gdiff>0', 'gdiff2']
                    #тогда целевой тпеременно будет только Gt
                    m_target = g_target.copy()
                    #и притворимся, что gt - это медиаданные 
                    #(это нужно для корректного выведения данных в дэшборде)
                    m = g.copy()
                else:
                    flag=False

            #Так, если у нас ничего не сломалось, то строим предсказание
            if flag:  
                x.columns = feature_names
                x = x.replace([np.inf, -np.inf], np.nan)
                x = x.fillna(method='ffill')
                x = x.astype(float).dropna()

                #Севастополь - особенный город... 
                #у него до 16 октября 2018 года почему-то беда с данными
                #на всякий случай уберем их руками
                if reg=='Севастополь':
                    last_year = datetime.fromisoformat('2018-10-16')
                    x = x[x.index>last_year]

                y = m_target.astype(int).copy()                
                y = y[y.index.isin(x.index)]
                
                #Важным является момент проверки совпадения датасетов при обучении и при построении результата.
                #Так как модели обучаются заново редко, то при накоплении данных, может оказаться, что
                #какой-то источник данных стал более или менее значимым и 
                #состав датасета при этом изменился.
                #Поэтому нужно проверять, что в нашем новом датасете есть все те признаки, 
                #на которых строился датасет для обучения,
                #а также обрезать лишние, если таковые возникли.
                with open(f'models/{colnames}', 'rb') as handle:
                    colnames = pickle.load(handle)
                    
                if not set(colnames).issubset(set(feature_names)):
                    print(f'Для региона {reg} не хватает признаков, рекомендуется проверить актуальность моделей.')
                    break
                else:
                    if not set(feature_names).issubset(set(colnames)):
                        print(f'Для региона {reg} есть лишние признаки./n' +\
                              'Предсказание будет построено, однако, рекомендуется проверить актуальность моделей.')
                    
                    x = x[colnames]
                    
                    y_pred_i = []
                    for est in estimators:
                        with open(f'models/{est}', 'rb') as handle:
                            est = pickle.load(handle)                    
                        y_pred_i.append(est.predict(x))
                    with open(f'models/{final_reg}', 'rb') as handle:
                            regr = pickle.load(handle)  
                    y_pred = regr.predict(pd.DataFrame(y_pred_i).T)
                    y_pred = pd.Series(y_pred, index=x.index)
                    #отнормируем на единицу
                    y_pred = y_pred/y_pred.max()

                    #Это очень аккуратная попытка учесть годовую сезонность.
                    #Проблема заключается в том, что не для всех регионов сезонность годовая,
                    #а на промежутке в 3 года это может означать, что сезонности на самом деле нет.
                    #Это вполне возможно, так как мы предсказываем в том числе заболеваемость,
                    #А она зависи не столько от календаря, сколько от того, какая стоит погода
                    #и где какой-нибудь герой перезаражал полшколы.
                    if 'delta' in list(x.columns):
                        #Смотрим, есть ли годовая сезонность
                        autocorr = mmx.long_term_correlation(m.shift(360), m, w1=60, w2=60, diff_w=60)
                        #Если мы смогли ее вычислить, несмотря на пробелы в данных
                        if not np.isnan(autocorr):
                            #Эта магическая опреация делает следующее:
                            #наша бинарная перменная указывает на дни, ПОСЛЕ КОТОРЫХ
                            #в течение недели будет подъем, а не самы подъемы.
                            #так что этой операцией мы заполняем эти подъемы,
                            #Тоже не самым корректным образом, но лучше, чем было
                            y_to_add = (y|mmx.find_range_of_holes(y, (1,7)))
                            #А теперь оставим только максимумы длиной не меньше месяца.
                            #Это не очень тривиально, но если вдуматься...
                            y_to_add = y_to_add&(~mmx.find_range_of_holes(~y_to_add, (1, 30)))
                            #Если мы уверена в нашей сезонности, то прибавим 0.4 к результату 
                            #и почти наверняка сделаем его триггером
                            if autocorr>0.5:
                                y_pred_added = y_pred + 0.4*(y_to_add.shift(360, fill_value=0))
                            #Если у нас есть сомнения, то добавим чуть-чуть
                            else:
                                y_pred_added = y_pred + 0.2*(y_to_add.shift(360, fill_value=0)&y_to_add.shift(720, fill_value=0))
                            #Ну и обрежем
                            y_pred = y_pred_added.apply(lambda x: min(x, 1))
                    predictions[reg] = pd.concat([y_pred, y, m], axis=1)
                    predictions[reg].columns = ['y_pred', 'y', 'youscan']  
                    predictions[reg].index = [i+timedelta(days=1) for i in predictions[reg].index]
                    predictions[reg] = predictions[reg][predictions[reg].index<=(datetime.now() + timedelta(hours=3))]
                
    return predictions

In [11]:
predictions_1 = region_prediction(weeks=1)
predictions_2 = region_prediction(weeks=2)

In [12]:
predictions_1_ = copy.deepcopy(predictions_1)
predictions_2_ = copy.deepcopy(predictions_2)

Достроим регионы с нехваткой данных по соседним

In [ ]:
predictions_1 = copy.deepcopy(predictions_1_)
predictions_2 = copy.deepcopy(predictions_2_)
nb = pd.read_csv('DATA/neighbour-regions.csv')
n_dict = {}
for col in nb.columns:
    n_dict[col] = nb[col].dropna().to_list()
#Этот словарь основан
neighbour_dict = {
    'Якутия': ['Хабаровский край'],
    'Ханты-Мансийский АО — Югра': ['Томская область', 'Тюменская область'],
    'Тыва': ['Красноярский край'],
    'Калмыкия': ['Ставропольский край'],
    'Ненецкий АО': ['Коми', 'Архангельская область'],
    'Еврейская АО': ['Хабаровский край'],
    'Севастополь': ['Крым'],
    'Амурская область': ['Хабаровский край'],
    'Дагестан': ['Ставропольский край'],
    'Ингушетия': ['Ставропольский край'],
    'Кабардино-Балкария': ['Ставропольский край'],
    'Магаданская область': ['Хабаровский край'],
    'Северная Осетия — Алания': ['Ставропольский край'],
    'Чукотский АО': ['Камчатский край'],
    'Крым': ['Севастополь']
}

for reg in neighbour_dict:
    n_dict[reg] = neighbour_dict[reg]

ind = predictions_1['Ярославская область'].dropna().index
#обработка регионов без прогноза
for reg in sorted(list(geo['Регион'].unique())):
    if reg not in list(predictions_1.keys()):
        print(reg)
        if len(n_dict[reg])>1:            
            predictions_1[reg] = [predictions_1[nr] for nr in n_dict[reg]]
            predictions_1[reg] = reduce(lambda x, y: x.add(y, fill_value=0), predictions_1[reg])/len(n_dict[reg])
        else: 
            predictions_1[reg] = predictions_1[n_dict[reg][0]]
    if reg not in list(predictions_2.keys()):
        if len(n_dict[reg])>1:            
            predictions_2[reg] = [predictions_2[nr] for nr in n_dict[reg]]
            predictions_2[reg] = reduce(lambda x, y: x.add(y, fill_value=0), predictions_2[reg])/len(n_dict[reg])
        else: 
            predictions_2[reg] = predictions_2[n_dict[reg][0]]

In [ ]:
#Раньше gt был сжат до недель и приходилось его растягивать. 
#Теперь он растягивается сразу и все это не надо, но код пусть останется.

# for reg in ['Сахалинская область', 'Камчатский край', 
#             'Карачаево-Черкесия', 'Дагестан', 'Тыва', 'Адыгея',
#            'Чукотский АО', 'Крым']:
#     a = predictions_1[reg].index[0]
#     predictions_1[reg] = pd.concat([predictions_1[reg]]*7).sort_index()
#     numdays = len(predictions_1[reg])
#     dateList = []
#     for x in range (0, numdays):
#         dateList.append(a + timedelta(x))
#     predictions_1[reg].index = dateList  
#     today = datetime.today()
#     predictions_1[reg] = predictions_1[reg][predictions_1[reg].index<today]
    
    
#     a = predictions_2[reg].index[0]
#     predictions_2[reg] = pd.concat([predictions_2[reg]]*7).sort_index()     
#     numdays = len(predictions_2[reg])
#     dateList = []
#     for x in range (0, numdays):
#         dateList.append(a + timedelta(x))
#     predictions_2[reg].index = dateList      
#     today = datetime.today()
#     predictions_2[reg] = predictions_2[reg][predictions_2[reg].index<today] 

In [ ]:
#Создадим финальную таблицу

ok_pred = {}
for reg in predictions_1:
    predictions_1[reg]['y'][-7:] = np.nan
    if isinstance(predictions_1[reg], pd.DataFrame):
        predictions_1[reg]['region'] = reg
        ok_pred[reg] = predictions_1[reg]
    else:
        ok_pred[reg] = pd.DataFrame(index=ind)
        ok_pred[reg]['region'] = reg
        ok_pred[reg]['y_pred'] = np.nan                           
        ok_pred[reg]['youscan'] = np.nan   
    last_year = datetime.today() - timedelta(days=370)
    ok_pred[reg] = ok_pred[reg][ok_pred[reg].index>last_year]
final_df_1 = pd.concat(ok_pred.values(), sort=True)
final_df_1 = final_df_1.reset_index()
final_df_1 = final_df_1.rename(columns={'y_pred': 'y_pred_1',
                                        'y': 'target_1week',
                                        'index': 'date'})

ok_pred_2 = {}
for reg in predictions_2:
    predictions_2[reg]['y'][-14:] = np.nan
    if isinstance(predictions_2[reg], pd.DataFrame):
        predictions_2[reg]['region'] = reg
        ok_pred_2[reg] = predictions_2[reg]
    else:
        ok_pred_2[reg] = pd.DataFrame(index=ind)
        ok_pred_2[reg]['region'] = reg
        ok_pred_2[reg]['y_pred'] = np.nan                         
        ok_pred_2[reg]['youscan'] = np.nan
    last_year = datetime.today() - timedelta(days=370)
    ok_pred_2[reg] = ok_pred_2[reg][ok_pred_2[reg].index>last_year]
final_df_2 = pd.concat(ok_pred_2.values(), sort=True)
final_df_2 = final_df_2.reset_index() 
final_df_2 = final_df_2.rename(columns={'y_pred': 'y_pred_2',
                                        'y': 'target_2week',
                                        'index': 'date'})
final_df = final_df_1.merge(final_df_2, on=['date', 'region'], how='outer')

In [ ]:
#категоризируем значения
final_df.index = final_df['date']
intervals = pd.IntervalIndex.from_tuples([(0, 0.4), (0.4, 0.8), (0.8, 1.1)], closed='left')
labels = dict(zip(['[0.0, 0.4)', '[0.4, 0.8)', '[0.8, 1.1)', 'nan'],
                  ['0', '1', '2', np.nan]))                  
final_df['y_pred_1_int'] = pd.cut(final_df['y_pred_1'], intervals).astype(str).replace(labels).astype(float)
final_df['y_pred_2_int'] = pd.cut(final_df['y_pred_2'], intervals).astype(str).replace(labels).astype(float)
final_df = final_df.drop(columns=['date', 'y_pred_1', 'y_pred_2', 'youscan_y'])
final_df = final_df.rename(columns={'youscan_x': 'youscan'})
final_df = final_df.rename(columns={'y_pred_1': 'predicted_1week', 
                                    'y_pred_2': 'predicted_2week' })
final_df = final_df[['region', 'youscan', 'target_1week', 'target_2week', 'y_pred_1_int', 'y_pred_2_int']]

In [ ]:
today = (datetime.now() + timedelta(hours=3)).date().strftime('%Y-%m-%d')
final_df.to_csv(f'predictions/trigger_{today}.csv')